In [ ]:
import requests
import json
# URL do dataset
dataset_url = "https://huggingface.co/datasets/spacemanidol/product-search-corpus/resolve/main/corpus-simple.jsonl"

# Baixar o dataset
response = requests.get(dataset_url)
data = [json.loads(line) for line in response.text.splitlines()]

# Converter para DataFrame
df = pd.DataFrame(data)

# Exibir as primeiras linhas do DataFrame para verificar a estrutura
print(df.head())

In [ ]:
from openai import OpenAI
import pandas as pd

# Configurar a chave da API do OpenAI
client = OpenAI(api_key='---')

In [ ]:
df = df.drop(columns = ['docid'])
df['title_text'] = df.apply(lambda row: f"{row['title']} / {row['text']}", axis=1)
df

In [ ]:
from openai import OpenAI



# Configurar a chave da API do OpenAI
client = OpenAI(api_key='---')

def extract_features_from_openai(text):

    prompt = (
        f"Extraia as principais features do seguinte texto:\n\n{text}\n\n"
        "Inclua apenas as features: categoria, material, handmade, cor, marca, me retorne em json com nome features com array de objetos.\n\n"
        "Inclua apenas um JSON e um array de objetos na resposta.\n\n. Caso a feature nao seja encontrada sempre escreva Não encontrado.\n\n se tiver mais de uma resposta utilize virgulas para separar apenas"
        "A resposta deve seguir ESTRITAMENTE a seguinte forma, igual sempre, não mude nada no formato: \n\n"
        "{\n"
        '  "features": [\n'
        '    {\n'
        '      "categoria": "resposta",\n'
        '      "material": "resposta",\n'
        '      "Público-alvo": "resposta(infantil, adulto, profissional, etc.)",\n'
        '      "cor": "resposta",\n'
        '      "marca": "resposta"\n'
        '    }\n'
        '  ]\n'
        "}"
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "Você é um assistente que ajuda a extrair features de descrições de produtos.Siga estritamente o comando de output dado no prompt sempre."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.6,
    )
    return response.choices[0].message.content.strip()

# Extrair features para o título e a descrição
dk['features'] = dk['title_text'].apply(lambda x: extract_features_from_openai(x) if print(f"Processing row {dk.index[dk['title_text'] == x][0]}") is None else None)
#df['title_features'] = df['title'].apply(lambda x: extract_features_from_openai(x))
#df['description_features'] = df['description'].apply(lambda x: extract_features_from_openai(x) if print(f"Processing row {df.index[df['title'] == x][0]}") is None else None)

In [ ]:
import pandas as pd
import json

# Supondo que 'dk' seja o seu DataFrame
def clean_json_prefix(data):
    if data.startswith('```json\\n'):
        return data.replace('```json\\n', '', 1)
    return data

# Limpar o prefixo '```json\n' das linhas
dk['features'] = dk['features'].apply(clean_json_prefix)

def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except ValueError:
        return False

# Filtrando as linhas que possuem JSON válido
valid_json_mask = dk['features'].apply(is_valid_json)
valid_data = dk[valid_json_mask]

# Aplicando json.loads apenas nos dados válidos
data_solved = valid_data['features'].apply(json.loads)

# Expandindo a coluna 'features' em colunas separadas
features_df = pd.json_normalize(data_solved, 'features')

# Concatenando o dataframe original com as novas colunas
dk_final = pd.concat([valid_data.drop(columns=['features']), features_df], axis=1)

# Opcional: lidar com as linhas que possuem JSON inválido
invalid_data = dk[~valid_json_mask]

In [ ]:
valid_data.to_csv('/workspace/GABRYEL_ALENCAR_DDF_TECH_072024/data/raw/chatgpt3-5.csv', index= False)

In [ ]:
dk_final = dk_final[0:998]
dk_final.drop(columns = ['handmade'], inplace = True)

In [ ]:
dk_final.to_csv('/workspace/GABRYEL_ALENCAR_DDF_TECH_072024/data/processed/Item 5/dados_LLM.csv', index = False)

In [ ]:
dk = df[0:100001]

In [ ]:
import os
from groq import Groq

# Configurar a chave da API do Groq
client = Groq(api_key="---")

def extract_features_from_groq(text):

    prompt = (
        f"Extraia as principais features do seguinte texto:\n\n{text}\n\n"
        "Inclua apenas as features: categoria, material, handmade, cor, marca, me retorne em json com nome features com array de objetos.\n\n"
        "Inclua apenas um JSON e um array de objetos na resposta.\n\n. Caso a feature nao seja encontrada sempre escreva Não encontrado.\n\n se tiver mais de uma resposta utilize virgulas para separar apenas"
        "A resposta deve seguir ESTRITAMENTE a seguinte forma, igual sempre, não mude nada no formato: \n\n"
        "{\n"
        '  "features": [\n'
        '    {\n'
        '      "categoria": "resposta",\n'
        '      "material": "resposta",\n'
        '      "Público-alvo": "resposta(infantil, adulto, profissional, etc.)",\n'
        '      "cor": "resposta",\n'
        '      "marca": "resposta"\n'
        '    }\n'
        '  ]\n'
        "}"
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Você é um assistente que ajuda a extrair features de descrições de produtos. Siga estritamente o comando de output dado no prompt sempre."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama3-8b-8192",
        temperature=0.6,
        max_tokens=100,

    )
    return chat_completion.choices[0].message.content.strip()

dk['features'] = dk['title_text'].apply(lambda x: extract_features_from_groq(x) if print(f"Processing row {dk.index[dk['title_text'] == x][0]}") is None else None)
